In [1]:
# Ensure pycryptodome is installed
try:
    import Crypto
except ModuleNotFoundError:
    %pip install pycryptodome

In [1]:
import sys
sys.path.append(r"../util")

In [2]:
import kis_auth as ka
import kis_domstk as kb

import os
import time
import logging
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

In [3]:
ka.auth()

In [6]:

hhmmss = datetime.strptime('200111', "%H%M%S").strftime("%H%M%S").rjust(10, "0")

In [ ]:
res = kb.get_news_title(date_1='20250413', hour_1=hhmmss)

AttributeError: module 'kis_domstk' has no attribute 'get_news_title'

: 

In [ ]:
# Set paths and configure logging to save to a file
destdir = r'E:\data\kis\news'
os.makedirs(destdir, exist_ok=True)

log_file_path = os.path.join(destdir, 'news_processing.log')
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Set date range
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 3, 23)

# Generate a list of dates
dates = [(start_date + timedelta(days=x)).strftime('%Y%m%d') for x in range((end_date - start_date).days + 1)]
dates.sort(reverse=True)

# Columns to drop
dropcols = ['iscd2', 'iscd3', 'iscd4', 'iscd5', 'iscd6', 
            'iscd7', 'iscd8', 'iscd9', 'iscd10', 
            'kor_isnm2', 'kor_isnm3', 'kor_isnm4', 
            'kor_isnm5', 'kor_isnm6', 'kor_isnm7', 
            'kor_isnm8', 'kor_isnm9', 'kor_isnm10']

for yyyymmdd in dates:
    yyyymmdd = yyyymmdd.rjust(10, "0")
    hhmmss = "0000001000"
    lstdf = []

    # Progress bar
    pbar = tqdm(total=0, unit="iteration", dynamic_ncols=True)

    while True:
        try:
            # Fetch news titles
            news_titles_df = kb.get_news_titles(date_1=yyyymmdd, hour_1=hhmmss)
            if len(news_titles_df) > 0:
                lstdf.append(pd.DataFrame(news_titles_df))
            
            # Update time
            hhmmss_datetime = datetime.strptime(hhmmss[-6:], "%H%M%S") + timedelta(seconds=10)
            
            if hhmmss_datetime >= datetime.strptime("235900", "%H%M%S"):
                break
            
            hhmmss = hhmmss_datetime.strftime("%H%M%S").rjust(10, "0")
            time.sleep(1)
            pbar.set_description(f'date = {yyyymmdd[2:]}, hour = {hhmmss[4:]}')
            pbar.update(1)

        except Exception as e:
            logging.error(f"Error fetching data for {yyyymmdd} at {hhmmss}: {e}")
            break

    pbar.close()

    if lstdf:
        try:
            # Concatenate data frames and save to file
            df = pd.concat(lstdf, ignore_index=True)
            df.drop(dropcols, axis=1, inplace=True)
            file_path = os.path.join(destdir, f'news_titles_{yyyymmdd[2:]}.feather')
            df.to_feather(file_path)
            logging.info(f"Data saved to {file_path}")
        except Exception as e:
            logging.error(f"Error processing data for {yyyymmdd}: {e}")
    else:
        logging.info(f"No data available for {yyyymmdd}")

# Output log file path if needed
logging.info(f"Logs saved to: {log_file_path}")

0iteration [00:00, ?iteration/s]

date = 20240323, hour = 171010: : 6121iteration [3:10:37,  1.87s/iteration]
date = 20240322, hour = 235850: : 8573iteration [4:28:06,  1.88s/iteration]
date = 20240321, hour = 235850: : 8573iteration [4:26:38,  1.87s/iteration]
date = 20240320, hour = 034120: : 1268iteration [39:20,  1.86s/iteration]

KeyboardInterrupt: 